# Pair Selenium

### Ejercicios Selenium Extracción de Datos de Libros de Ficción en La Casa del Libro

El código resultado está al final de todo. Antes se realizan varias pruebas:

In [11]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.
# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests
# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.
# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [12]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from selenium.common.exceptions import TimeoutException, NoSuchElementException

In [13]:
url = 'https://www.casadellibro.com/?utm_source=Bing&utm_medium=cpc&utm_campaign=marca&utm_content=153&campaignid=710730986&msclkid=16bbdce23c51170794f3b5effd84d8d8'

Para el pop-up del registro que sólo sale a veces:

In [22]:
# ============================
# Función para cerrar popups
# ============================

def cerrar_popups(driver):
    wait = WebDriverWait(driver, timeout=10)  # Se define aquí el wait

    # Aceptar cookies si aparece
    try:
        btn_cookies = wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "#onetrust-accept-btn-handler"))
        )
        btn_cookies.click()
        print("Cookies aceptadas")
    except TimeoutException:
        print("Popup de cookies no apareció")

    # Cerrar popup de login si aparece
    try:
        btn_cerrar_login = driver.find_element(By.CSS_SELECTOR, "body > dialog > div > div > div.d-flex.pa-1 > button").click()
        btn_cerrar_login.click()
        print("Popup login cerrado")
    except NoSuchElementException:
        print("No apareció popup de login")

In [15]:
driver = webdriver.Chrome()           # Creamos este objeto para poder interactuar con el navegador

driver.get(url)
print('Entro en la página')
driver.maximize_window()
print('Maximizo la ventana')
sleep(5)  # Buena práctica, darle tiempo al driver para entrar en la página, si no hará el click en el objeto siguiente y puede que todavía no exista prque no se haya cargado.
driver.find_element('css selector', '#onetrust-accept-btn-handler').click()  # Encuentra este elemento
print('Cookies aceptadas')
sleep(5)
driver.find_element("css selector", "#m-c > div:nth-child(2) > a").click() # Encuentra el elemento Ficción
print('Click libros Ficción')
sleep(5)


Entro en la página
Maximizo la ventana
Cookies aceptadas
Click libros Ficción


Selector 1r y 2o libro de la página Ficción:

In [ ]:
#App > div.cma-components.container.d-grid.svelte-f3gp65 > cma-component:nth-child(3) > div > div.swiper.svelte-1x4dp7n > div.roller.svelte-1x4dp7n > div:nth-child(2) > a > div > cdl-img > img
#App > div.cma-components.container.d-grid.svelte-f3gp65 > cma-component:nth-child(3) > div > div.swiper.svelte-1x4dp7n > div.roller.svelte-1x4dp7n > div:nth-child(3) > a > div > cdl-img > img

In [55]:
datos_libros = {'titulo':[], 'autora':[], 'precio':[], 'editorial':[], 'idioma':[], 'numero_paginas': []}

for i in range(1,6) :
    print(f'Accediendo al producto {i}')
    driver.find_element("css selector", f"#App > div.cma-components.container.d-grid.svelte-f3gp65 > cma-component:nth-child(3) > div > div.swiper.svelte-1x4dp7n > div.roller.svelte-1x4dp7n > div:nth-child({i+1}) > a > div > cdl-img > img").click() # Click en los diferentes libros
    sleep(5)

    titulo = driver.find_element("css selector", "#t-p-f").text # Sacamos el Título
    print('Extracción Titulo libro')

    autora = driver.find_element("css selector", "#a-p-f > p > a").text # Sacamos el autor/a
    print('Extracción Autor/a')

    precio = driver.find_element("css selector", "#p-pf-f").text # Sacamos el precio
    print('Extracción Precio')

    try:
        editorial = driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(1)").text.split('\n')[1]    # Sacamos editorial, si tiene traductor delante, será el child(2)      
    except:
        editorial = driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(2)").text.split('\n')[1]   
    print('Extracción Editorial')

    try:
        if editorial == driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(1)").text.split('\n')[1] :
           idioma = driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(3)").text.split('\n')[1]
        else:
            idioma = driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(4)").text.split('\n')[1]    # Sacamos el idioma 
    except:
        idioma = driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(4)").text.split('\n')[1]
    print('Extracción Idioma')

    try:
        numero_paginas = driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > div:nth-child(4)").text.split('\n')[1]    # Sacamos núm páginas
    except:
        numero_paginas = driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > div:nth-child(5)").text.split('\n')[1]  
    print('Extracción Número Páginas')

    datos_libros['titulo'].append(titulo)
    datos_libros['autora'].append(autora)
    datos_libros['precio'].append(precio)
    datos_libros['editorial'].append(editorial)
    datos_libros['idioma'].append(idioma)
    datos_libros['numero_paginas'].append(numero_paginas)
    
    driver.back()
    sleep(5)


Accediendo al producto 1
Extracción Titulo libro
Extracción Autor/a
Extracción Precio
Extracción Editorial
Extracción Idioma
Extracción Número Páginas
Accediendo al producto 2
Extracción Titulo libro
Extracción Autor/a
Extracción Precio
Extracción Editorial
Extracción Idioma
Extracción Número Páginas
Accediendo al producto 3
Extracción Titulo libro
Extracción Autor/a
Extracción Precio
Extracción Editorial
Extracción Idioma
Extracción Número Páginas
Accediendo al producto 4
Extracción Titulo libro
Extracción Autor/a
Extracción Precio
Extracción Editorial
Extracción Idioma
Extracción Número Páginas
Accediendo al producto 5
Extracción Titulo libro
Extracción Autor/a
Extracción Precio
Extracción Editorial
Extracción Idioma
Extracción Número Páginas


In [56]:
datos_libros

{'titulo': ['VERA, UNA HISTORIA DE AMOR (PREMIO PLANETA 2025)',
  'LOS TRES MUNDOS (SERIE JULIO CÉSAR 3)',
  'ISLAS DE LA ASCUAOSCURA',
  'ALFA',
  'LA PENÍNSULA DE LAS CASAS VACÍAS'],
 'autora': ['Juan del Val',
  'Santiago Posteguillo',
  'Brandon Sanderson',
  'Ali Hazelwood',
  'David Uclés'],
 'precio': ['20,80 €', '24,60 €', '25,55 €', '18,95 €', '24,70 €'],
 'editorial': ['Editorial Planeta',
  'B',
  'Nova',
  'Faeris Editorial',
  'Siruela'],
 'idioma': ['Castellano',
  'Castellano',
  'Castellano',
  'Castellano',
  'Castellano'],
 'numero_paginas': ['360', '1040', '560', '480', '700']}

In [ ]:
driver.find_element("css selector", "#App > div.cma-components.container.d-grid.svelte-f3gp65 > cma-component:nth-child(3) > div > div.swiper.svelte-1x4dp7n > div.roller.svelte-1x4dp7n > div:nth-child(2) > a > div > cdl-img > img").click() # Click en el primer libro
#print('Click 1r libro Ficción')
sleep(5)
driver.find_element("css selector", "#t-p-f").text # Sacamos el Título
print('Extracción Titulo libro')
driver.find_element("css selector", "#a-p-f > p > a").text # Sacamos el autor/a
print('Extracción Autor/a')
driver.find_element("css selector", "#p-pf-f").text # Sacamos el precio
print('Extracción Precio')
driver.find_element("css selector", "#App > div.cma-components.container.d-grid.svelte-f3gp65 > cma-component:nth-child(1) > div > area-3 > p").text # Sacamos editorial
print('Extracción Editorial')
driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(3)").text.split('\n')[1]    # Sacamos el idioma 
print('Extracción Idioma')
driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > div:nth-child(4)").text.split('\n')[1]    # Sacamos núm páginas  
print('Extracción Número Páginas')

'20,80 €'

In [ ]:
driver.find_element("css selector", "#t-p-f").text # Título

'VERA, UNA HISTORIA DE AMOR (PREMIO PLANETA 2025)'

In [ ]:
driver.find_element("css selector", "#a-p-f > p > a").text # autor/a

'Juan del Val'

In [ ]:
driver.find_element("css selector", "#p-pf-f").text # precio

'20,80 €'

In [ ]:
driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(1)").text.split('\n')[1]              # editorial
                                    #contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(2)   # si en la ficha técnica tiene info del Traductor --> child(2)


'Editorial Planeta'

In [ ]:
try:
    editorial = driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(1)").text.split('\n')[1]           # editorial
except:
    editorial = driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(2)").text.split('\n')[1]

In [ ]:
driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(3)").text.split('\n')[1]    # idioma  
                                    #contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(3)
                                    #contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(4)
                                    #contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(3)

'Castellano'

In [24]:
driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > div:nth-child(4)").text.split('\n')[1]    # núm páginas 

'360'

CÓDIGO FINAL COMPLETO:

In [26]:
driver = webdriver.Chrome()           # Creamos este objeto para poder interactuar con el navegador

driver.get(url)
print('Entro en la página')
driver.maximize_window()
print('Maximizo la ventana')
sleep(5)  # Buena práctica, darle tiempo al driver para entrar en la página, si no hará el click en el objeto siguiente y puede que todavía no exista prque no se haya cargado.
cerrar_popups(driver)  # cookies y registro
sleep(5)
driver.find_element("css selector", "#m-c > div:nth-child(2) > a").click() # Encuentra el elemento Ficción
print('Click libros Ficción')
sleep(5)


datos_libros = {'titulo':[], 'autora':[], 'precio':[], 'editorial':[], 'idioma':[], 'numero_paginas': []}

for i in range(1,6) :
    print(f'Accediendo al producto {i}')
    driver.find_element("css selector", f"#App > div.cma-components.container.d-grid.svelte-f3gp65 > cma-component:nth-child(3) > div > div.swiper.svelte-1x4dp7n > div.roller.svelte-1x4dp7n > div:nth-child({i+1}) > a > div > cdl-img > img").click() # Click en los diferentes libros
    sleep(7)

    titulo = driver.find_element("css selector", "#t-p-f").text # Sacamos el Título
    print('Extracción Titulo libro')

    autora = driver.find_element("css selector", "#a-p-f > p > a").text # Sacamos el autor/a
    print('Extracción Autor/a')

    precio = driver.find_element("css selector", "#p-pf-f").text # Sacamos el precio
    print('Extracción Precio')

    try:
        editorial = driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(1)").text.split('\n')[1]    # Sacamos editorial, si tiene traductor delante, será el child(2)      
    except:
        editorial = driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(2)").text.split('\n')[1]   
    print('Extracción Editorial')

    try:
        if editorial == driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(1)").text.split('\n')[1] :
           idioma = driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(3)").text.split('\n')[1]
        else:
            idioma = driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(4)").text.split('\n')[1]    # Sacamos el idioma 
    except:
        idioma = driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > h3:nth-child(4)").text.split('\n')[1]
    print('Extracción Idioma')

    try:
        numero_paginas = driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > div:nth-child(4)").text.split('\n')[1]    # Sacamos núm páginas
    except:
        numero_paginas = driver.find_element("css selector", "#contributors > div.mt-4.caracteristicas.svelte-1yhk452 > div:nth-child(5)").text.split('\n')[1]  
    print('Extracción Número Páginas')

    datos_libros['titulo'].append(titulo)
    datos_libros['autora'].append(autora)
    datos_libros['precio'].append(precio)
    datos_libros['editorial'].append(editorial)
    datos_libros['idioma'].append(idioma)
    datos_libros['numero_paginas'].append(numero_paginas)
    
    driver.back()
    sleep(5)

print('Extracción finalizada')
driver.close()

Entro en la página
Maximizo la ventana
Cookies aceptadas
No apareció popup de login
Click libros Ficción
Accediendo al producto 1
Extracción Titulo libro
Extracción Autor/a
Extracción Precio
Extracción Editorial
Extracción Idioma
Extracción Número Páginas
Accediendo al producto 2
Extracción Titulo libro
Extracción Autor/a
Extracción Precio
Extracción Editorial
Extracción Idioma
Extracción Número Páginas
Accediendo al producto 3
Extracción Titulo libro
Extracción Autor/a
Extracción Precio
Extracción Editorial
Extracción Idioma
Extracción Número Páginas
Accediendo al producto 4
Extracción Titulo libro
Extracción Autor/a
Extracción Precio
Extracción Editorial
Extracción Idioma
Extracción Número Páginas
Accediendo al producto 5
Extracción Titulo libro
Extracción Autor/a
Extracción Precio
Extracción Editorial
Extracción Idioma
Extracción Número Páginas
Extracción finalizada


In [27]:
inventario_libros = pd.DataFrame(datos_libros)

In [28]:
inventario_libros

,titulo,autora,precio,editorial,idioma,numero_paginas
0,"VERA, UNA HISTORIA DE AMOR (PREMIO PLANETA 2025)",Juan del Val,"20,80 €",Editorial Planeta,Castellano,360
1,LOS TRES MUNDOS (SERIE JULIO CÉSAR 3),Santiago Posteguillo,"24,60 €",B,Castellano,1040
2,ISLAS DE LA ASCUAOSCURA,Brandon Sanderson,"25,55 €",Nova,Castellano,560
3,ALFA,Ali Hazelwood,"18,95 €",Faeris Editorial,Castellano,480
4,LA PENÍNSULA DE LAS CASAS VACÍAS,David Uclés,"24,70 €",Siruela,Castellano,700
